In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import requests
import shutil
import bs4
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
import pyautogui
import cv2
import time

In [2]:
#import excel that contains tiktok link
df_link = pd.read_excel('link tiktok.xlsx')

In [17]:
PATH = "C:\chromedriver_win32\chromedriver.exe"
driver = webdriver.Chrome(PATH)

#empty list
links = []
title = []
username = []
likes = []
views = []
comment = []
share = []

#ambil url yang ingin di scrape
for i in range(len(df_link.iloc[:,0])):
    link = df_link.iloc[i,0]

    #open and maximize window
    driver.get(link)
    driver.maximize_window()

    time.sleep(6)

    #CAPTCHA SOLVER
    #download image function
    def down(image, xpath):
        #get link image
        captcha = driver.find_element (By.XPATH, xpath)
        src = captcha.get_attribute('src')

        #download image
        response = requests.get(src, stream=True)
        with open(image, 'wb') as out_file:
            shutil.copyfileobj(response.raw, out_file)
        del response

    #download captcha image
    down('captcha_img.jpeg', "//*[@id='captcha-verify-image']")
    #download piece of captcha image
    down('captcha_piece.jpeg', "//*[@id='tiktok-verify-ele']/div/div[2]/img[2]")

    time.sleep(2)

    #read img
    img = cv2.imread('captcha_img.jpeg')
    img_piece = cv2.imread('captcha_piece.jpeg')

    #CONTOURS FUNCTION
    def bin_img(image):
        img = cv2.imread(image)
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        binary_img = cv2.threshold(img_gray, 242, 255, cv2.THRESH_BINARY_INV)[1]

        return binary_img

    capt = bin_img('captcha_img.jpeg')
    piece = bin_img('captcha_piece.jpeg')

    #matching shape
    match = cv2.matchTemplate(capt, piece, cv2.TM_CCOEFF_NORMED)

    #check x, y in maximum value
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(match)

    #width and height
    w = piece.shape[1]
    h = piece.shape[0]

    time.sleep(2)

    #drag slider with mouse movement
    #gerakin mouse ke slider button
    pyautogui.moveTo(782, 722, duration=1.5)
    #drag mouse ke puzzle pece yang benar
    pyautogui.dragRel(0.78*max_loc[0], 0, button='left', duration=2)

    time.sleep(5)

    #exit login bar
    ext_login_bar = driver.find_element(By.XPATH, '//*[@id="login-modal"]/div[2]')
    ext_login_bar.click()

    time.sleep(2)

    #start and pause the video
    pyautogui.moveTo(367, 1002, duration=1)
    pyautogui.click(button='left')
    time.sleep(2)
    pyautogui.click(button='left')

    time.sleep(2)

    #ADD USERNAME, LIKES, COMMENT, SHARE
    #find username
    xpath_username= '//*[@id="main-content-video_detail"]/div/div[2]/div[1]/div[2]/div/a[2]/span[1]'
    link_username = driver.find_element(By.XPATH, xpath_username)
    get_link_username = link_username.text
    #find likes
    xpath_likes = '//*[@id="main-content-video_detail"]/div/div[2]/div[1]/div[1]/div[1]/div[4]/div/button[1]/strong'
    link_likes = driver.find_element(By.XPATH, xpath_likes)
    get_link_likes = link_likes.text
    #find comment
    xpath_comment = '//*[@id="main-content-video_detail"]/div/div[2]/div[1]/div[1]/div[1]/div[4]/div/button[2]/strong'
    link_comment = driver.find_element(By.XPATH, xpath_comment)
    get_link_comment = link_comment.text
    #find share
    xpath_share = '//*[@id="main-content-video_detail"]/div/div[2]/div[1]/div[1]/div[1]/div[4]/div/button[3]/strong'
    link_share = driver.find_element(By.XPATH, xpath_share)
    get_link_share = link_share.text

    #adding info to list
    username.append(get_link_username)
    likes.append(get_link_likes)
    comment.append(get_link_comment)
    share.append(get_link_share)

    #open profile
    profile = driver.find_element(By.XPATH, '//*[@id="main-content-video_detail"]/div/div[2]/div[1]/div[2]/div/a[2]/span[1]')
    profile.click()

    time.sleep(2)

    #scroll to the bottom
    previous_height = driver.execute_script('return document.body.scrollHeight')

    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
        time.sleep(3)
        new_height = driver.execute_script('return document.body.scrollHeight')
        
        if new_height == previous_height:
            break
        previous_height = new_height

    #scroll to the top
    driver.execute_script('window.scrollTo(0, 0)')

    #take the html source
    page_html = driver.page_source
    page_soup = bs4.BeautifulSoup(page_html, 'html.parser')
    #find all content
    content = page_soup.find_all('div', {'class':"tiktok-x6y88p-DivItemContainerV2 e19c29qe7"})

    len_content = len(content)

    #looping didalem profile mereka sampai ketemu link yang sama
    for j in range(1, len_content+1):
        #find url
        xpath_url = f'//*[@id="main-content-others_homepage"]/div/div[2]/div[2]/div/div[{j}]/div[1]/div/div/a'
        link_url = driver.find_element(By.XPATH, xpath_url)
        get_link_url = link_url.get_attribute('href')

        if get_link_url != link:
            continue
        #find views
        xpath_views = f'//*[@id="main-content-others_homepage"]/div/div[2]/div[2]/div/div[{j}]/div[1]/div/div/a/div/div[2]/strong'
        link_views = driver.find_element(By.XPATH, xpath_views)
        get_link_views = link_views.text
        #find title
        xpath_title = f'//*[@id="main-content-others_homepage"]/div/div[2]/div[2]/div/div[{j}]/div[2]/a'
        link_title = driver.find_element(By.XPATH, xpath_title)
        get_link_title = link_title.get_attribute('title')

        #adding info
        links.append(get_link_url)
        title.append(get_link_title)
        views.append(get_link_views)

        break

    #exit chrome
    driver.quit()
        
        

#Result
df_result = pd.DataFrame(list(zip(link, title, username, likes, views, comment, share)), columns=['link', 'title', 'username', 'likes', 'views', 'comment', 'share'])

#exit chrome
#driver.quit()

C:\Users\Public\Documents\iSkysoft\CreatorTemp\ipykernel_35832\375753429.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="main-content-others_homepage"]/div/div[2]/div[2]/div/div[1]/div[1]/div/div/a"}
  (Session info: chrome=113.0.5672.127)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x01158893+48451]
	(No symbol) [0x010EB8A1]
	(No symbol) [0x00FF5058]
	(No symbol) [0x01020467]
	(No symbol) [0x0102069B]
	(No symbol) [0x0104DD92]
	(No symbol) [0x0103A304]
	(No symbol) [0x0104C482]
	(No symbol) [0x0103A0B6]
	(No symbol) [0x01017E08]
	(No symbol) [0x01018F2D]
	GetHandleVerifier [0x013B8E3A+2540266]
	GetHandleVerifier [0x013F8959+2801161]
	GetHandleVerifier [0x013F295C+2776588]
	GetHandleVerifier [0x011E2280+612144]
	(No symbol) [0x010F4F6C]
	(No symbol) [0x010F11D8]
	(No symbol) [0x010F12BB]
	(No symbol) [0x010E4857]
	BaseThreadInitThunk [0x750F6BA9+25]
	RtlGetFullPathName_UEx [0x77318F9F+1215]
	RtlGetFullPathName_UEx [0x77318F6D+1165]


In [ ]:
#export data to excel
df_result.to_excel('tiktok data scrape result.xlsx', index=False)